## Creating a Genetic algorithm

In [ ]:
import pandas as pd
from datetime import date
from sqlalchemy import create_engine, text



In [3]:


connection_string = (
    "mssql+pyodbc:///?odbc_connect="
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-PGFBDFD\SQLEXPRESS;"
    "DATABASE=AlgeriaUspaV3;"
    "Trusted_Connection=yes;"
)

engine = create_engine(connection_string)
df = pd.read_sql("SELECT TOP 10 * FROM ProductAttributes", engine)
print(df)

   ItemTypeCode   ItemCode ProductAtt01 ProductAtt02 ProductAtt03  \
0             1  100021356           01           01            S   
1             1  100021357           01           01            S   
2             1  100021358           01           01            S   
3             1  100021359           01           01            S   
4             1  100021360           01           01            S   
5             1  100021361           01           01            S   
6             1  100021362           01           02            S   
7             1  100021363           01           02            S   
8             1  100021364           01           02            S   
9             1  100021365           01           02            S   

  ProductAtt04 ProductAtt05 ProductAtt06 ProductAtt07 ProductAtt08  ...  \
0                         1                      2020       WINTER  ...   
1                         1                      2020       WINTER  ...   
2              

In [10]:
df = pd.read_sql("SELECT TOP 50 * FROM dbo.RetailSales WHERE OperationDate BETWEEN '2024-04-19' AND '2024-04-21'", engine)
print(df)

    CurrAccTypeCode CurrAccCode OperationDate OperationTime DocumentNumber  \
0                 4    1-4-5903    2024-04-20      19:20:45    1-R-7-26194   
1                 4       1-4-1    2024-04-19      11:01:28    1-R-7-26034   
2                 4       1-4-1    2024-04-19      11:07:39    1-R-7-26035   
3                 4       1-4-1    2024-04-20      19:24:45    1-R-7-26196   
4                 4       1-4-1    2024-04-20      19:24:45    1-R-7-26196   
5                 4       1-4-1    2024-04-20      19:41:43    1-R-7-26199   
6                 4       1-4-1    2024-04-20      19:41:43    1-R-7-26199   
7                 4       1-4-1    2024-04-20      20:02:57    1-R-7-26201   
8                 4       1-4-1    2024-04-20      20:22:37    1-R-7-26203   
9                 4       1-4-1    2024-04-20      21:03:45    1-R-7-26206   
10                4       1-4-1    2024-04-20      21:03:45    1-R-7-26206   
11                4       1-4-1    2024-04-20      17:58:04    1

NameError: name 'conn_str' is not defined

In [12]:
# === Config ===
store_code = 'CHE'
warehouse_code = 'PRI'
start_date = '2024-04-19'
end_date = '2024-04-20'
plan_date = '2024-04-21'

In [63]:
name_query = text("""
SELECT
    ItemCode,
    ItemDescription
FROM dbo.cdItemDesc                 
""")
name_df = pd.read_sql(name_query, engine)
print(name_df)

       ItemCode ItemDescription
0     100021356           HY001
1     100021357           HY001
2     100021358           HY001
3     100021359           HY001
4     100021360           HY001
...         ...             ...
6087       ExD2     Kur Farkı 3
6088        TZM        Tazminat
6089         VF      Vade Farkı
6090        VF1    Vade Farkı 1
6091        VF2    Vade Farkı 2

[6092 rows x 2 columns]


In [59]:
barcode_query = text("""
SELECT 
    ItemCode,
    ColorCode,
    ItemDim1Code,
    Barcode
FROM dbo.prItemBarcode  
""")
barcode_df = pd.read_sql(barcode_query, engine)
print(barcode_df)

        ItemCode ColorCode ItemDim1Code        Barcode
0      100021356      C044           40  8600000000011
1      100021356      C044           41  8600000000028
2      100021356      C044           42  8600000000035
3      100021356      C044           43  8600000000042
4      100021356      C044           44  8600000000059
...          ...       ...          ...            ...
58679     YP8186         1           56  8681896033765
58680     YP8186         1           58  8681896033789
58681      ZERDA     VR033            0  8682107842800
58682      ZERDA     VR041            0  8682107842824
58683      ZERDA     VR071            0  8682107842817

[58684 rows x 4 columns]


In [40]:

# === 1. Get sales from the store between the two dates ===
sales_sql = text("""
SELECT 
    ItemCode,
    ColorCode,
    ItemDim1Code,
    SUM(CASE WHEN IsReturn = 1 THEN -Qty1 ELSE Qty1 END) AS NetQty
FROM dbo.RetailSales
WHERE StoreCode = :store
  AND OperationDate BETWEEN :start AND :end
  AND Qty1 > 0
GROUP BY ItemCode, ColorCode, ItemDim1Code
HAVING SUM(CASE WHEN IsReturn = 1 THEN -Qty1 ELSE Qty1 END) > 0
""")
sales_df = pd.read_sql(sales_sql, engine, params={
    'store': store_code,
    'start': start_date,
    'end': end_date
})
print(sales_df, engine)


      ItemCode ColorCode ItemDim1Code  NetQty
0    101109663      C074           22     1.0
1    101346103     C0240           37     1.0
2    101389883      C044           37     1.0
3    101389982      C042           39     1.0
4    101390115     C0249           36     1.0
..         ...       ...          ...     ...
58  USB1307-V3     C0425           17     1.0
59  USB1307-V4     C0421           17     1.0
60   USB904-V1     C0131        18_24     1.0
61   USB904-V3     C0144        24_36     1.0
62    Y23AD015      C080          115     1.0

[63 rows x 4 columns] Engine(mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BODBC+Driver+17+for+SQL+Server%7D%3BSERVER%3DDESKTOP-PGFBDFD%5CSQLEXPRESS%3BDATABASE%3DAlgeriaUspaV3%3BTrusted_Connection%3Dyes%3B)


In [41]:
# --- Get warehouse available inventory ---
stock_sql = text("""
SELECT 
    ItemCode,
    ColorCode,
    ItemDim1Code,
    AvailableInventoryQty1
FROM dbo.ItemInventory
WHERE WarehouseCode = :wh
""")
stock_df = pd.read_sql(stock_sql, engine, params={'wh': warehouse_code})
print(stock_df, engine)

        ItemCode ColorCode ItemDim1Code  AvailableInventoryQty1
0      100021356      C044           41                     4.0
1      100021356      C044           42                     1.0
2      100021356      C044           43                     1.0
3      100021357      C074           40                     1.0
4      100021357      C074           41                     6.0
...          ...       ...          ...                     ...
4988    Y23AS012      C080          130                    -3.0
4989  Y23AS019-C     C0197          110                     1.0
4990  Y23AS019-C     C0197          115                     1.0
4991  Y23AS019-C     C0197          125                     0.0
4992  Y23AS019-C     C0197          130                     0.0

[4993 rows x 4 columns] Engine(mssql+pyodbc:///?odbc_connect=DRIVER%3D%7BODBC+Driver+17+for+SQL+Server%7D%3BSERVER%3DDESKTOP-PGFBDFD%5CSQLEXPRESS%3BDATABASE%3DAlgeriaUspaV3%3BTrusted_Connection%3Dyes%3B)


In [42]:
pri = pd.read_sql("SELECT TOP 50 * FROM dbo.ItemInventory WHERE WarehouseCode LIKE 'PRI' ", engine)
print(pri)

    CompanyCode OfficeCode  StoreTypeCode StoreCode WarehouseCode  \
0           1.0          M              5                     PRI   
1           1.0          M              5                     PRI   
2           1.0          M              5                     PRI   
3           1.0          M              5                     PRI   
4           1.0          M              5                     PRI   
5           1.0          M              5                     PRI   
6           1.0          M              5                     PRI   
7           1.0          M              5                     PRI   
8           1.0          M              5                     PRI   
9           1.0          M              5                     PRI   
10          1.0          M              5                     PRI   
11          1.0          M              5                     PRI   
12          1.0          M              5                     PRI   
13          1.0          M        

In [43]:
merged = pd.merge(
    sales_df,
    stock_df,
    on=['ItemCode', 'ColorCode', 'ItemDim1Code'],
    how='inner'
)
merged['ToTransfer'] = merged[['NetQty', 'AvailableInventoryQty1']].min(axis=1)
plan_df = merged[merged['ToTransfer'] > 0].copy()

In [44]:
plan_df['StoreID'] = store_code
plan_df['PlanDate'] = plan_date
plan_df['SourceWarehouse'] = warehouse_code
plan_df.rename(columns={'ToTransfer': 'Quantity'}, inplace=True)

result = plan_df[['StoreID', 'ItemCode', 'ColorCode', 'ItemDim1Code', 'Quantity', 'PlanDate', 'SourceWarehouse']]


In [46]:
print(result)

   StoreID   ItemCode ColorCode ItemDim1Code  Quantity    PlanDate  \
0      CHE  101109663      C074           22       1.0  2024-04-21   
4      CHE  101391915     C0397           41       1.0  2024-04-21   
5      CHE  101391918      C015           42       1.0  2024-04-21   
6      CHE  101392294      C058           38       1.0  2024-04-21   
9      CHE  101399383      C038           43       1.0  2024-04-21   
10     CHE  101416461      C038           38       1.0  2024-04-21   
11     CHE  101416461      C038           39       1.0  2024-04-21   
12     CHE    1362021     VR093            M       1.0  2024-04-21   
15     CHE    1461184     VR049            S       1.0  2024-04-21   
16     CHE    1570603     VR046           36       1.0  2024-04-21   
18     CHE    1571222     VR033            L       1.0  2024-04-21   
20     CHE    1573851     VR015            L       1.0  2024-04-21   
21     CHE    1573851     VR015            M       1.0  2024-04-21   
22     CHE    162965

In [53]:
result_with_barcode = pd.merge(
    result,
    barcode_df,
    on=['ItemCode', 'ColorCode', 'ItemDim1Code'],
    how='left'  # Use left join to preserve all replenishment rows
)


In [66]:
result_with_name = pd.merge(
    result_with_barcode,
    name_df,
    on = ['ItemCode'],
    how = 'left'
)
print(result_with_name)

   StoreID   ItemCode ColorCode ItemDim1Code  Quantity    PlanDate  \
0      CHE  101109663      C074           22       1.0  2024-04-21   
1      CHE  101391915     C0397           41       1.0  2024-04-21   
2      CHE  101391918      C015           42       1.0  2024-04-21   
3      CHE  101392294      C058           38       1.0  2024-04-21   
4      CHE  101399383      C038           43       1.0  2024-04-21   
5      CHE  101416461      C038           38       1.0  2024-04-21   
6      CHE  101416461      C038           39       1.0  2024-04-21   
7      CHE    1362021     VR093            M       1.0  2024-04-21   
8      CHE    1461184     VR049            S       1.0  2024-04-21   
9      CHE    1570603     VR046           36       1.0  2024-04-21   
10     CHE    1571222     VR033            L       1.0  2024-04-21   
11     CHE    1573851     VR015            L       1.0  2024-04-21   
12     CHE    1573851     VR015            M       1.0  2024-04-21   
13     CHE    162965

In [54]:
print(result_with_barcode)

   StoreID   ItemCode ColorCode ItemDim1Code  Quantity    PlanDate  \
0      CHE  101109663      C074           22       1.0  2024-04-21   
1      CHE  101391915     C0397           41       1.0  2024-04-21   
2      CHE  101391918      C015           42       1.0  2024-04-21   
3      CHE  101392294      C058           38       1.0  2024-04-21   
4      CHE  101399383      C038           43       1.0  2024-04-21   
5      CHE  101416461      C038           38       1.0  2024-04-21   
6      CHE  101416461      C038           39       1.0  2024-04-21   
7      CHE    1362021     VR093            M       1.0  2024-04-21   
8      CHE    1461184     VR049            S       1.0  2024-04-21   
9      CHE    1570603     VR046           36       1.0  2024-04-21   
10     CHE    1571222     VR033            L       1.0  2024-04-21   
11     CHE    1573851     VR015            L       1.0  2024-04-21   
12     CHE    1573851     VR015            M       1.0  2024-04-21   
13     CHE    162965

In [ ]:
# Export to Excel
result_with_name.to_excel(
    "C:/Users/ACER/Downloads/replenishment_CHE_apr19_20.xlsx",
    sheet_name='ReplenishmentPlan',
    index=False
)
